# Imports

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

pd.set_option('display.max_columns', None)

# Datensätze

In [2]:
matches = pd.read_csv('../../data/raw/atp_matches_till_2022.csv')
players = pd.read_csv('../../data/raw/atp_players_till_2022.csv')
rankings = pd.read_csv('../../data/raw/atp_rankings_till_2022.csv')

# Fragestellung 3
### Gibt es Turniere, bei denen die Deutschen besonders gut oder besonders schlecht abschneiden? Wenn ja, ist ein regionales Muster oder eine andere Ursache zu erkennen?

In [3]:
# Filtern von Spielen mit deutschen Spielern
german_player_matches = matches[(matches['winner_ioc'] == 'GER') | (matches['loser_ioc'] == 'GER')]
german_player_matches['tourney_name'] = german_player_matches['tourney_name'].str.replace(r'Davis Cup.*', 'Davis Cup', regex=True)
german_player_matches['tourney_name'] = german_player_matches['tourney_name'].str.replace(r' 1| 2|-2|WCT| WCT|- WCT', '', regex=True)

# Zählen der Siege und Niederlagen für deutsche Spieler in jedem Turnier
tournament_performance = german_player_matches.groupby('tourney_name').apply(
    lambda x: pd.Series({
        'Wins': (x['winner_ioc'] == 'GER').sum(),
        'Losses': (x['loser_ioc'] == 'GER').sum()
    })
).reset_index()

# Berechnung der Gewinnquote
tournament_performance['Win_Rate'] = tournament_performance['Wins'] / (tournament_performance['Wins'] + tournament_performance['Losses'])

# Filtern von Turnieren mit mindestens 5 Spielen mit deutscher Beteiligung
tournaments_with_min_matches = tournament_performance[tournament_performance['Wins'] + tournament_performance['Losses'] >= 5]

# Sortieren nach Gewinnquote
tournaments_with_min_matches_sorted_by_win_rate = tournaments_with_min_matches.sort_values(by='Win_Rate', ascending=False)



/var/folders/jq/xvfn2vg55lsc1wx2_2mrh3640000gn/T/ipykernel_15721/866837215.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  german_player_matches['tourney_name'] = german_player_matches['tourney_name'].str.replace(r'Davis Cup.*', 'Davis Cup', regex=True)
/var/folders/jq/xvfn2vg55lsc1wx2_2mrh3640000gn/T/ipykernel_15721/866837215.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  german_player_matches['tourney_name'] = german_player_matches['tourney_name'].str.replace(r' 1| 2|-2|WCT| WCT|- WCT', '', regex

In [4]:
# Erstellen Sie ein Dictionary mit den Turniernamen und den entsprechenden Ländern
tourney_country = {
    'Indian Wells': 'USA',
    'Laver Cup': 'CAN',
    'Charleston': 'USA',
    'Aix-en-Provence': 'FRA',
    'Chicago': 'USA',
    'Livingston': 'GB-SCB',
    'Oslo': 'NOR',
    'Masters Dec': '',
    'Boca Raton': 'USA',
    'Omaha': 'USA',
    'Vina del Mar': 'CHL',
    'Quito': 'ECU',
    'Key Biscayne': 'USA',
    'Los Cabos': 'MEX',
    'Buzios': 'BRA',
    'Johannesburg': 'ZAF',
    'Aix en Provence': 'FRA',
    'Chengdu': 'CHN',
    'Perth': 'AUS',
    'Melbourne': 'AUS'
}

# Erstellen Sie eine neue Spalte 'country' basierend auf der 'tourney_name' Spalte
tournaments_with_min_matches_sorted_by_win_rate['country'] = tournaments_with_min_matches_sorted_by_win_rate['tourney_name'].map(tourney_country)

In [5]:
# Anzeigen der besten und schlechtesten Turniere basierend auf der Gewinnquote
top_performing_tournaments_with_min_matches = tournaments_with_min_matches_sorted_by_win_rate.sort_values(by=['Win_Rate', 'Wins'], ascending=False).head(10)
worst_performing_tournaments_with_min_matches = tournaments_with_min_matches_sorted_by_win_rate.sort_values(by=['Win_Rate', 'Losses'], ascending=False).tail(10)

### Top 5 Turniere, bei denen die Deutschen am besten abgeschnitten haben

In [6]:
top_performing_tournaments_with_min_matches

,tourney_name,Wins,Losses,Win_Rate,country
118,Indian Wells,12,2,0.857143,USA
139,Laver Cup,6,1,0.857143,CAN
64,Charleston,5,1,0.833333,USA
3,Aix-en-Provence,5,1,0.833333,FRA
68,Chicago,9,2,0.818182,USA
143,Livingston,8,2,0.800000,GB-SCB
204,Oslo,4,1,0.800000,NOR
167,Masters Dec,4,1,0.800000,
37,Boca Raton,7,2,0.777778,USA
200,Omaha,9,3,0.750000,USA


In [7]:
fig = px.choropleth(top_performing_tournaments_with_min_matches, locations=top_performing_tournaments_with_min_matches["country"], color="Win_Rate", range_color=[0,1], title="Länder der Turniere, in denen Deutschland gut abgeschnitten hat.")
fig.show()

Man kann erkennen, dass Deutschland vorallem auf Turnieren im englischsprachigen Raum bzw. gut abschneidet.

### Top 5 Turniere, bei denen die Deutschen am schlechtesten abgeschnitten haben

In [8]:
worst_performing_tournaments_with_min_matches

,tourney_name,Wins,Losses,Win_Rate,country
299,Vina del Mar,3,8,0.272727,CHL
222,Quito,3,8,0.272727,ECU
128,Key Biscayne,7,19,0.269231,USA
150,Los Cabos,2,6,0.250000,MEX
54,Buzios,2,7,0.222222,BRA
127,Johannesburg,2,7,0.222222,NaN
2,Aix en Provence,2,8,0.200000,FRA
66,Chengdu,1,4,0.200000,CHN
215,Perth,0,5,0.000000,AUS
169,Melbourne,0,5,0.000000,AUS


In [9]:
fig = px.choropleth(worst_performing_tournaments_with_min_matches, 
                    locations=worst_performing_tournaments_with_min_matches["country"], 
                    color="Win_Rate", range_color=[0,1], 
                    title="Länder der Turniere, in denen Deutschland schlecht abgeschnitten hat.")
fig.show()

- Deutschland hat ein gemäßigtes Klima mit relativ milden Sommern und Wintern. Turniere in Ländern wie Australien, Chile, oder Brasilien finden oft in viel wärmeren oder feuchteren Bedingungen statt, was eine erhebliche Anpassung von den Spielern erfordert.
- Extreme Hitze oder hohe Luftfeuchtigkeit können die körperliche Belastung während eines Spiels erhöhen. Spieler, die nicht an solche Bedingungen gewöhnt sind, können schneller ermüden, was ihre Leistung beeinträchtigen kann.

Der Power Bi Report für Aufgabe 3 ist an die sportliche Leitung des deutschen Tennisbundes:

Um die Leistung deutscher Tennisspieler in internationalen Turnieren zu verbessern, insbesondere in Ländern mit ungewohnten Klimabedingungen, könnten folgende Vorschläge hilfreich sein:

- Frühere Anreise zu Turnieren: Die Spieler sollten rechtzeitig vor Turnierbeginn anreisen, um sich an das lokale Klima, die Zeitzone und die Umgebung anzupassen. Dies gibt ihnen Zeit, sich zu akklimatisieren und sich mental und physisch auf die bevorstehenden Spiele vorzubereiten.
- Physiologische und psychologische Unterstützung: Bereitstellung von Experten wie Sportpsychologen und Ernährungsberatern, die den Spielern helfen können, sich physisch und mental auf die Herausforderungen der verschiedenen Turniere vorzubereiten.
- Spezifische Fitnessprogramme: Entwicklung von Fitnessprogrammen, die auf die Anforderungen des jeweiligen Turniers zugeschnitten sind, einschließlich Ausdauertraining für wärmere Klimazonen und Übungen zur Verbesserung der allgemeinen Beweglichkeit und Schnelligkeit.
- Taktische Anpassungen: Die Trainer sollten spezifische Spielstrategien entwickeln, die an die Bedingungen und Spielflächen der verschiedenen Turniere angepasst sind. Dazu kann auch die Analyse der Gegner gehören, um maßgeschneiderte Spielpläne zu erstellen.